In [713]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [714]:
class SplitInfo:
    
        def __init__(self, feature_name, threshold):
            self.feature_name = feature_name
            self.threshold = threshold
            self.details = None
            self.metric = None
    
class ISplitStrategy:    
    def get_recomendation(self, features : pd.DataFrame, target: pd.Series) -> SplitInfo:
        pass

class GiniImpuritySplitStrategy(ISplitStrategy):
    
    def get_recommendation(self, features : pd.DataFrame, target: pd.Series, strategy, split_num=None) -> SplitInfo:

        split_result = pd.DataFrame(columns=['feature_name', 'threshold', 'gini_impurity_index'])
        if split_num == None:
            split_number = 5
        else:
            split_number = split_num
        
        for feature_name in features.columns:
            
            feature = features[feature_name]
           
            if strategy == 'median':
                thresholds = [feature.median()]
            else:
                bins = np.linspace(.1, 1, split_number, 0)
                thresholds = feature.quantile(bins)
            
            for threshold in thresholds:
                
                ft_df = FeatureTargetDataFrame(feature, target)

                dataset_split = ft_df.split(split_method='threshold', split_value=threshold)
                subset1 = dataset_split.subset1
                subset2 = dataset_split.subset2
            
                subset1_gini_index = self.gini_impurity_index(subset1[target.name].value_counts())
                subset2_gini_index = self.gini_impurity_index(subset2[target.name].value_counts())

                gini_index = (subset1_gini_index + subset2_gini_index) / 2
            
                split_result.loc[len(split_result)] = [feature_name, dataset_split.threshold, gini_index]
            
        min_gini_index = split_result['gini_impurity_index'].min()
        filt = split_result['gini_impurity_index'] == min_gini_index

        split_by = split_result.loc[filt]
        
        split_by_feature_name = split_by['feature_name'].values[0]
        split_by_threshold = split_by['threshold'].values[0]
        
        split_info = SplitInfo(split_by_feature_name, split_by_threshold)
        split_info.details = split_result
        split_info.metric = min_gini_index

        return split_info
            
    
    def is_split_required(self, target):
        gini_index = self.gini_impurity_index(target.value_counts())
        if(gini_index > 0.01):
            return True
        return False
    
    def gini_impurity_index (self, value_counts):
        n = value_counts.sum()
        p_sum = 0
        for key in value_counts.keys():
            p_sum = p_sum  +  (value_counts[key] / n ) * (value_counts[key] / n ) 
        gini = 1 - p_sum
        return gini

In [715]:
class DatasetSplit:
    
    def __init__(self):
        self.threshold = None
        self.subset1 = None
        self.subset2 = None

class FeatureTargetDataFrame:
    
    def __init__(self, feature, target):
        
        self.feature_name = feature.name
        self.target_name = target.name
        
        self.df = pd.DataFrame()
        self.df[feature.name] = feature
        self.df[target.name] = target

    def get_greater(self, threshold):
        
        filt = self.df[self.feature_name] > threshold
        subset = self.__get_by_filter(filt)
        return subset
    
    def get_less_or_equal(self, threshold):
        
        filt = self.df[self.feature_name] <= threshold
        subset = self.__get_by_filter(filt)
        return subset
    
    def split(self, split_method, split_value=None):
        
        if(split_method != 'median' and split_method != 'threshold'):
            raise Exception(f'Unsupported split method {split_method}')
        
        if split_method == 'median':
            threshold = self.df[self.feature_name].median()
        else:
            threshold = split_value
            
        subset1 = self.get_greater(threshold)
        subset2 = self.get_less_or_equal(threshold)
        
        dataset_split = DatasetSplit()
        dataset_split.threshold = threshold
        dataset_split.subset1 = subset1
        dataset_split.subset2 = subset2
        
        return dataset_split
    
    def __get_by_filter(self, filt):
        
        subset = pd.DataFrame()
        columns = self.df.columns
        subset[columns] = self.df.loc[filt][columns].copy()
        return subset

In [716]:
class DecisionTree:
    
    class Node:
        
        def __init__(self, features, target, parent, strategy):
           
            self.features = features.copy()
            self.target = target.copy()
            self.parent = parent
            self.strategy = strategy
            self.left = None
            self.right = None
            self.feature_name = None
            self.threshold = None
            self.splitted = False
            self.metric = None
            
            if(parent == None):
                self.height = 0
            else:
                self.height = parent.height + 1
        
        def print_node(self):
            
            if self.splitted:
                print(f'Node({str(round(self.metric, 2))} ({str(self.feature_name)} -> {str(self.threshold)}))')
                self.left.print_node()
                self.right.print_node()
        
        def split(self):
            
            #if self.height > 15:
            #    return
            
            is_split_required = self.strategy.is_split_required(self.target)
            
            if is_split_required:
                recommendation = self.strategy.get_recommendation \
                    (self.features, self.target, strategy='quantile', split_num=3)
                
                threshold = recommendation.threshold
                feature_name = recommendation.feature_name
                
                df = pd.DataFrame()
                df[self.features.columns] = self.features.copy()
                df[self.target.name] = self.target.copy()
                
                filt = df[feature_name] > threshold
                subset = df.loc[filt]
                subset_features = subset[self.features.columns]
                subset_target = subset[self.target.name]
                
                self.left = DecisionTree.Node \
                    (subset_features, subset_target, self, GiniImpuritySplitStrategy())
                
                filt = df[feature_name] <= threshold
                subset = df.loc[filt]
                subset_features = subset[self.features.columns]
                subset_target = subset[self.target.name]
                
                self.right = DecisionTree.Node \
                    (subset_features, subset_target, self, GiniImpuritySplitStrategy())
                
                self.threshold = threshold
                self.feature_name = feature_name
                self.metric = recommendation.metric
                
                self.left.split()
                self.right.split()
                
                self.splitted = True

    class Tree:
        
        def __init__(self):
            self.root = None
            self.nodes = []
    
    class Classifier(Tree):
            
        def get_height(self):
            return self.root.height
        
        def print_tree(self):
            self.root.print_node()
        
        def fit(self, features, target):
            
            root = DecisionTree.Node(features, target, None, GiniImpuritySplitStrategy())
            self.root = root
            self.root.split()
            
            self.__append_node(self.root)
            
        def __append_node(self, node):
            self.nodes.append(node)
            
            if not(node.left is None):
                self.__append_node(node.left)
                
            if not(node.right is None):
                self.__append_node(node.right)
            

## Decision tree test

## Loading dataset

In [717]:
report_df = pd.read_csv('../datasets/other/world_happiness_report.csv')
report_df.head()

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440


In [718]:
renamed_columns = {
    'Country Name' : 'country',
    'Regional Indicator' : 'region',
    'Year' : 'year',
    'Life Ladder' : 'life_ladder',
    'Log GDP Per Capita' : 'gdp_per_capita',
    'Social Support' : 'social_support',
    'Healthy Life Expectancy At Birth' : 'healthy_life',
    'Freedom To Make Life Choices' : 'free_to_choose',
    'Generosity' : 'generosity',
    'Perceptions Of Corruption' : 'corruption',
    'Positive Affect' : 'pos_affect',
    'Negative Affect' : 'neg_affect',
    'Confidence In National Government' : 'gov_confidence'
}

report_df.rename(columns=renamed_columns, inplace=True)
report_df.head()

,country,region,year,life_ladder,gdp_per_capita,social_support,healthy_life,free_to_choose,generosity,corruption,pos_affect,neg_affect,gov_confidence
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440


In [719]:
report_df.drop(columns=['country', 'generosity', 'corruption', 'year', \
    'pos_affect', 'neg_affect', 'gov_confidence', 'healthy_life', 'gdp_per_capita'], inplace=True)
report_df.head()

,region,life_ladder,social_support,free_to_choose
0,South Asia,3.723590,0.450662,0.718114
1,South Asia,4.401778,0.552308,0.678896
2,South Asia,4.758381,0.539075,0.600127
3,South Asia,3.831719,0.521104,0.495901
4,South Asia,3.782938,0.520637,0.530935


In [720]:
print(f'Dataset size BEFORE NaN row elimination is {str(len(report_df))}')

feature_median = report_df.social_support.median()
filt = report_df.social_support.isnull()
report_df.loc[filt, 'social_support'] =  feature_median

feature_median = report_df.free_to_choose.median()
filt = report_df.free_to_choose.isnull()
report_df.loc[filt, 'free_to_choose'] =  feature_median

report_df.dropna(inplace=True)

print(f'Dataset size AFTER NaN row elimination is {str(len(report_df))}')

Dataset size BEFORE NaN row elimination is 2199
Dataset size AFTER NaN row elimination is 2087


In [721]:
if (report_df.isnull().mean() == 0).all():
    print('Dataset NaN were values eliminated')

Dataset NaN were values eliminated


In [722]:
#Split code
def gini_impurity_index (value_counts):
    n = value_counts.sum()
    p_sum = 0
    for key in value_counts.keys():
        p_sum = p_sum  +  (value_counts[key] / n ) * (value_counts[key] / n ) 
    gini = 1 - p_sum
    return gini

feature = report_df['free_to_choose']
target = report_df['region']

ft_df = FeatureTargetDataFrame(feature, target)

dataset_split = ft_df.split(split_method='median')
subset1 = dataset_split.subset1
subset2 = dataset_split.subset2

print(f'Dataset lenght is {str(len(ft_df.df))}')
print(f'Subset 1 lenght is {str(len(subset1))}')
print(f'Subset 2 lenght is {str(len(subset2))}')

dataset_split.subset2.head()

Dataset lenght is 2087
Subset 1 lenght is 1043
Subset 2 lenght is 1044


,free_to_choose,region
0,0.718114,South Asia
1,0.678896,South Asia
2,0.600127,South Asia
3,0.495901,South Asia
4,0.530935,South Asia


In [723]:
subset1_gini_index = gini_impurity_index(subset1[target.name].value_counts())
subset2_gini_index = gini_impurity_index(subset2[target.name].value_counts())

gini_index = (subset1_gini_index + subset2_gini_index) / 2
gini_index

print(f'Subset 1 gini index is {str(subset1_gini_index)}')
print(f'Subset 2 gini index is {str(subset2_gini_index)}')
print(f'Avarage gini index is {str(gini_index)}')

Subset 1 gini index is 0.8586191649760215
Subset 2 gini index is 0.829749636675915
Avarage gini index is 0.8441844008259682


In [724]:
split_strategy = GiniImpuritySplitStrategy()
recommendation = split_strategy.get_recommendation \
    (report_df.drop(columns=['region']), report_df.region, strategy='median')

recommendation.details

,feature_name,threshold,gini_impurity_index
0,life_ladder,5.457299,0.803659
1,social_support,0.836828,0.821356
2,free_to_choose,0.770360,0.844184


In [725]:
split_strategy = GiniImpuritySplitStrategy()
recommendation = split_strategy.get_recommendation \
    (report_df.drop(columns=['region']), report_df.region, strategy='quantile', split_num=3)

recommendation.details

,feature_name,threshold,gini_impurity_index
0,life_ladder,4.036151,0.659016
1,life_ladder,5.138650,0.786740
2,life_ladder,6.136989,0.793278
3,social_support,0.638157,0.748525
4,social_support,0.809170,0.812851
5,social_support,0.895644,0.814521
6,free_to_choose,0.564422,0.845092
7,free_to_choose,0.735775,0.846645
8,free_to_choose,0.840221,0.834470


In [726]:
print(f'Best feature is "{recommendation.feature_name}"')

Best feature is "life_ladder"


In [727]:
print(f'Best theshold is "{recommendation.threshold}"')

Best theshold is "4.0361505507999995"


In [728]:
classifier = DecisionTree.Classifier()
classifier.fit(report_df.drop(columns=['region']), report_df.region)

classifier.print_tree()

Node(0.66 (life_ladder -> 4.0361505507999995))
Node(0.77 (life_ladder -> 4.4520866869999995))
Node(0.79 (life_ladder -> 6.354436159))
Node(0.61 (social_support -> 0.9401774644))
Node(0.35 (life_ladder -> 7.497455168))
Node(0.07 (life_ladder -> 7.506003618))
Node(0.08 (life_ladder -> 7.514631271))
Node(0.09 (life_ladder -> 7.561203718))
Node(0.1 (life_ladder -> 7.5797310826))
Node(0.11 (life_ladder -> 7.595542812))
Node(0.12 (social_support -> 0.9468459368))
Node(0.14 (social_support -> 0.9487955568))
Node(0.19 (life_ladder -> 7.7919274332))
Node(0.25 (life_ladder -> 7.7074011804))
Node(0.19 (free_to_choose -> 0.8575645922))
Node(0.12 (free_to_choose -> 0.9172298372000001))
Node(0.0 (free_to_choose -> 0.9340868948000001))
Node(0.51 (life_ladder -> 7.04073143))
Node(0.41 (life_ladder -> 7.1536588668))
Node(0.38 (life_ladder -> 7.4282414915))
Node(0.12 (life_ladder -> 7.4434130672))
Node(0.13 (life_ladder -> 7.4445368764))
Node(0.15 (life_ladder -> 7.4480993274))
Node(0.17 (free_to_choose

In [729]:
len(classifier.nodes)

2587